# Workshop 1 - Apply
    
In this notebook you will work on a homework assignment involving a Vierendeel frame.

Our matrix method implementation is now completely stored in a local package, consisting of three classes.

## Vierendeel frame

<center><figure>
  <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/vierendeel.png" WIDTH=400 ALIGN="center">
      </figure></center>

With:

- $h = 1$
- $b = 1$
- $EI_r = 10000$
- $EI_k = 1000$
- $EA  = 1\cdot 10^{10}$
- $H = 100$


In the first half of this course last quarter, you have learned that the deformation of Vierendeel frames (an example of which is shown above) can be obtained in a simplified way by assuming the global deformation can be described by a shear beam with equivalent stiffness given by:

$$
k = \frac{24}{h\left(\displaystyle\frac{h}{EI_k}+\frac{b}{EI_r}\right)}
$$

Now that you have the tools to solve the original frame problem using the Matrix Method, your task in this assignment is to investigate the validity of this equivalent shear beam model.

Note that the checks only had a single element. For this model you need to obtain $\mathbf{K}$ and $\mathbf{f}$ of all elements and add them to the correct locations in the global stiffness matrix and force vector. To do that, make use of the `global_dofs` function of the Element class and the `np.ix_` Numpy utility function. (Tip: refer back to what you did in the `constrain` function).

Once you have a solution, use SymPy / Maple / pen and paper to solve a shear beam problem with the equivalent stiffness given above (It is very similar to the simple extension problem above) and compare the horizontal displacement at the point of application of $H$ for the two models.

Investigate how the two models compare for different values of $EA$, ranging from very small (*e.g.* $1\cdot 10^{-5}$) to very large (*e.g.* $1\cdot10^{10}$). What explains the behavior you observe?

In [1]:
import numpy as np
import matrixmethod as mm
%config InlineBackend.figure_formats = ['svg']

In [2]:
mm.Node.clear()
mm.Element.clear()

In [3]:
h = 1
b = 1
EIr = 10000
EIk = 1000
EA = 1e10
H = 100

node1 = mm.Node(0, 0)
node2 = mm.Node(b, 0)
node3 = mm.Node(b, h)
node4 = mm.Node(0, h)

elem1 = mm.Element(node1, node2)
elem2 = mm.Element(node2, node3)
elem3 = mm.Element(node3, node4)
elem4 = mm.Element(node4, node1)

elem1.set_section({'EI': EIr, 'EA': EA})
elem2.set_section({'EI': EIk, 'EA': EA})
elem3.set_section({'EI': EIr, 'EA': EA})
elem4.set_section({'EI': EIk, 'EA': EA})

con = mm.Constrainer()

con.fix_dof(node1, 0)
con.fix_dof(node1, 1)
con.fix_dof(node2, 1)

node4.add_load([H, 0, 0])

In [4]:
elems = [elem1, elem2, elem3, elem4]
nodes = [node1, node2, node3, node4]
amount_of_nodes = len(nodes)


global_k = np.zeros((3 * amount_of_nodes, 3 * amount_of_nodes))
global_f = np.zeros(3 * amount_of_nodes)

i = 0

for elem in elems:
    elmat = elem.stiffness()
    idofs = elem.global_dofs()

    global_k[np.ix_(idofs, idofs)] += elmat

for node in nodes:
    global_f[node.dofs] += node.p

In [5]:
Kff, Fff = con.constrain(global_k, global_f)

u = np.linalg.inv(Kff) @ Fff
print(u)
print(u[-3])

[ 4.16667982e-04  4.99999539e-09  4.16665674e-04  4.58333833e-03
 -4.99999807e-09  4.16675352e-04  4.58334333e-03  4.99999807e-09
  4.16677659e-04]
0.004583343333683899


In [6]:
Hh = H

#provided in case you want to solve the shear beam problem using SymPy
import sympy as sym

x, k, L, H = sym.symbols('x, k, L, H')
w = sym.Function('w')

ODE_shear = sym.Eq(w(x).diff(x, 2) *k, 0)
w = sym.dsolve(ODE_shear, w(x)).rhs

gamma = w.diff(x)
V = k * gamma
eq1 = sym.Eq(w.subs(x,0),0)
eq2 = sym.Eq(V.subs(x,L),H)
C_sol = sym.solve([eq1, eq2], sym.symbols('C1, C2'))
display(w.subs(C_sol))

k = 24 / (h * (h / EIk + b / EIr))
print(Hh * h / k)

H*x/k

0.004583333333333333
